# Super Mario Bros RL Notebook
## Rohan H, Talaal S, Thang N, Yuet W
### University of Bath

![SegmentLocal](mario!.gif "segment")
(we are going to have to delete him sadly...)


### How to run:
The official website for the game environment can be found here: https://pypi.org/project/gym-super-mario-bros/

In a nutshell, you will need:
- Python 3.5/3.6/3.7/3.8 (I have tested on 3.7)
- gymnasium (gym is deprecated)
- ipykernel for running the notebook
- gym-super-mario-bros 
- other essential packages/libraries like NumPy
- an average computer

### Check if the environment works

In [1]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
env_example = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode='human')
env_example = JoypadSpace(env_example, SIMPLE_MOVEMENT)

test = False
done = True
if test:
    for step in range(1000):
        if done:
            obs, info = env_example.reset()
        obs, reward, terminated, truncated, info = env_example.step(env_example.action_space.sample())
        done = not terminated or truncated

env_example.close()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\gym\envs\registration.py:556: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  f"The environment {id} is out of date. You should consider "
c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\gym\envs\registration.py:628: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.f

# /// DQN Algorithm ///

## Importing Libraries, Defining Functions and Classes

In [ ]:
# Rohan's Cell

# YOUR CODE HERE
# example here: https://docs.pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

import math
import gc
import random
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import gym
from gym.spaces import Box
from gym.wrappers.frame_stack import FrameStack
from gym.core import ObservationWrapper
import gym_super_mario_bros.actions as actions

import torch
import torch.nn as nn
import torch.optim as opt
#import torch.nn.functional as func
#from torchvision import transforms as T

from collections import namedtuple, deque
from itertools import count
#import itertools

import cv2

def clean_mem():
    if torch.cuda.is_available():    
        torch.cuda.empty_cache()
    gc.collect()

def cuda():
    if torch.cuda.is_available():
        return True
    else: 
        return False
    
def device():
    if torch.cuda.is_available():    
        torch.cuda.empty_cache()
        return torch.device("cuda")
    
    else:
        return torch.device("cpu")

system = device()

Transition = namedtuple('Transition',
    ['state', 'action', 'next_state', 'reward'])


class ReplayMemory(object):
    
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)


class DQN_full(nn.Module):

    def __init__(self, shape, actions_n):
        super(DQN_full, self).__init__()
        self.convolution = nn.Sequential( # Convolutional layers
        nn.Conv2d(shape[0], 32, kernel_size=8, stride=4),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=4, stride=2),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, stride=1),
        nn.ReLU()
        )

        output_size = self._get_conv_out(shape)
        self.fully_connected = nn.Sequential( # 32 neurons -> 64 neurons -> "n" actions 
            nn.Linear(output_size, 128),
            nn.ReLU(),
            nn.Linear(128, actions_n)
        )
    def _get_conv_out(self, shape):
        with torch.no_grad():
            blank = torch.zeros(1, *shape)
            out = self.convolution(blank)
            return int(np.prod(out.size())) 

    def forward(self, x):
        out = self.convolution(x).view(x.size(0), -1)
        return self.fully_connected(out)
    

class Transpose(ObservationWrapper): # Put Greyscale channel as first argument
    def __init__(self, env):
        super().__init__(env)
        obs_shape = env.observation_space.shape
        self.observation_space = Box(
            low=self.observation_space.low.min(),
            high=self.observation_space.high.max(),
            shape=(obs_shape[2], obs_shape[0], obs_shape[1]),
            dtype=env.observation_space.dtype,
        )

    def observation(self, observation):
        return np.transpose(observation, (2, 0, 1))

class SmallBox84(ObservationWrapper): # Resize to 84x84
    def __init__(self, env):
        super().__init__(env)
        obs_shape = env.observation_space.shape
        self.observation_space = Box(
            low=0,
            high=255,
            shape=(84, 84, obs_shape[2]),
            dtype=np.uint8,
        )
    def observation(self, observation):
        observation = cv2.resize(observation, (84, 84), interpolation=cv2.INTER_AREA)
        if observation.ndim == 2:
            observation = observation[:, :, np.newaxis]
        return observation

class GreyScale(ObservationWrapper): # Convert to greyscale
    def __init__(self, env):
        super().__init__(env)
        obs_shape = env.observation_space.shape
        self.observation_space = Box(
            low=0,
            high=255,
            shape=(obs_shape[0], obs_shape[1], 1),
            dtype=np.uint8,
        )
    def transform(self, observation):
        return np.dot(observation[..., :3], [0.299, 0.587, 0.114])

    def observation(self, observation):
        observation = self.transform(observation)
        observation = observation.astype(np.uint8)
        observation = np.expand_dims(observation, axis=-1)
        return observation

class Normalise(ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = Box(
            low=0.0,
            high=1.0,
            shape=env.observation_space.shape,
            dtype=np.float32
        )
    def observation(self, observation):
        return observation.astype(np.float32) / 255.0

env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode='robot')
env = JoypadSpace(env, actions.SIMPLE_MOVEMENT)
env = GreyScale(env)
env = SmallBox84(env)
env = Transpose(env)
env = Normalise(env)

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\gym\envs\registration.py:556: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  f"The environment {id} is out of date. You should consider "
c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\gym\envs\registration.py:628: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  f"The environment creator metadata doesn't include `render_modes`, contains: {list(env_creator.metadata.keys())}"


In [3]:
gamma = 0.99 # discount reward
update = 0.0001 # learning rate
epsilon_start = 0.9 # initial exploration rate
epsilon_end = 0.01 # final exploration rate
max_steps = 1000 # step limit for decay
action_n = env.action_space.n
obs, info = env.reset()
eps_decay = True

network = DQN_full(obs.shape, action_n).to(system)

optimiser = opt.AdamW(network.parameters(), lr=update)
memory = ReplayMemory(5000) 
steps_passed = 0

def get_eps():
    if steps_passed >= max_steps:
        return epsilon_end
    else:
        return epsilon_start - (epsilon_start - epsilon_end) * (steps_passed / max_steps)

epsilon = 0.1
def select_action(state, network):
    global steps_passed
    sample = random.random()
    steps_passed += 1
    if eps_decay:
        epsilon = get_eps() 
    if sample > epsilon:
        with torch.no_grad():
            return network(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=system, dtype=torch.long)


## Model Optimiser

In [10]:
batch_size = 32
clean_mem()

def optimize_model():
    if len(memory) < batch_size: # take 32 of 5000 to update model params
        return
    transitions = memory.sample(batch_size)
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=system, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = network(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(batch_size, device=system)
    with torch.no_grad():
        next_state_values[non_final_mask] = network(non_final_next_states).max(1).values
    
    # Compute expected Q values
    expected_state_action_values = (next_state_values * gamma) + reward_batch

    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    optimiser.zero_grad()
    loss.backward()
    
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(network.parameters(), 15)
    optimiser.step()


## Training Loop

In [8]:
if cuda():
    episodes = 600
else:
    episodes = 30

episode_rewards = []
episode_steps = []

clean_mem()

for episode_n in range(episodes):
    print("Starting Episode ", episode_n+1)
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=system).unsqueeze(0)
    total_reward = 0
    done = None
    steps_passed = 0
    for i in range(20000): # or use count() until terminated
        action = select_action(state, network)
        obs, reward, terminated, truncated, info = env.step(action.item())
        total_reward += reward
        reward = torch.tensor([reward], device=system)
        done = terminated or truncated
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(obs, dtype=torch.float32, device=system).unsqueeze(0)

        memory.push(state, action, next_state, reward)
        state = next_state

        if i % 5 == 0:
            optimize_model()

        if i % 100 == 0:
            clean_mem()

        if done:
            break

    clean_mem()
        
    episode_rewards.append(total_reward)
    print(total_reward)

print('Done Training')

if cuda():
    torch.save(network.state_dict(), 'mario_DQN_full.pth')
else:
    torch.save(network.state_dict(), 'mario_DQN_short.pth')


Starting Episode  1
1437.0
Starting Episode  2
1851.0
Starting Episode  3
1544.0
Starting Episode  4
2656.0
Starting Episode  5
1505.0
Starting Episode  6
1722.0
Starting Episode  7
668.0
Starting Episode  8
1833.0
Starting Episode  9
1622.0
Starting Episode  10
1842.0
Starting Episode  11


KeyboardInterrupt: 

In [11]:
# Cleanup ready for DDQN
del memory, optimiser, network
clean_mem()

NameError: name 'network' is not defined

## /// DDQN ///

Since it is a variant of the DQN algorithm, some of the assets remain the same.


In [12]:
memory = ReplayMemory(5000)

class DDQN(nn.Module): # same as DQN class
    def __init__(self, shape, actions_n):
        super(DDQN, self).__init__()
        self.convolution = nn.Sequential( 
        nn.Conv2d(shape[0], 32, kernel_size=8, stride=4),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=4, stride=2),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, stride=1),
        nn.ReLU()
        )

        output_size = self._get_conv_out(shape)
        self.fully_connected = nn.Sequential( 
            nn.Linear(output_size, 128),
            nn.ReLU(),
            nn.Linear(128, actions_n)
        )
    def _get_conv_out(self, shape):
        with torch.no_grad():
            blank = torch.zeros(1, *shape)
            out = self.convolution(blank)
            return int(np.prod(out.size())) 

    def forward(self, x):
        out = self.convolution(x).view(x.size(0), -1)
        return self.fully_connected(out
        )

primary_network = DQN_full(obs.shape, action_n).to(system) 
target_network = DDQN(obs.shape, action_n).to(system)
target_network.load_state_dict(primary_network.state_dict())
target_network.eval()

optimiser = opt.AdamW(primary_network.parameters(), lr=update)

def optimize_ddqn():
    if len(memory) < batch_size: 
        return
    transitions = memory.sample(batch_size)
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=system, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = primary_network(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(batch_size, device=system)
    with torch.no_grad():
        if len(non_final_next_states) > 0:
            # Select best actions using the primary network
            q_values_primary = primary_network(non_final_next_states)
            best_actions = q_values_primary.argmax(1).unsqueeze(1)

            q_values_target = target_network(non_final_next_states)
            next_state_values[non_final_mask] = q_values_target.gather(1, best_actions).squeeze(1)

    expected_state_action_values = (next_state_values * gamma) + reward_batch

    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    optimiser.zero_grad()
    loss.backward()
    
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(primary_network.parameters(), 15)
    optimiser.step()


def optimise_target_network(tau=0.001):
    for target_param, primary_param in zip(target_network.parameters(), primary_network.parameters()):
        target_param.data.copy_(tau * primary_param.data + (1.0 - tau) * target_param.data)

### Training Loop for DDQN

In [13]:
if cuda():
    episodes = 600
else:
    episodes = 30

episode_rewards = []
episode_steps = []
clean_mem()

for episode_n in range(episodes):
    print("Starting Episode ", episode_n+1)
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=system).unsqueeze(0)
    total_reward = 0
    done = None
    steps_passed = 0
    for i in range(20000): # or use count() to go until terminated
        action = select_action(state, primary_network)
        obs, reward, terminated, truncated, info = env.step(action.item())
        total_reward += reward
        reward = torch.tensor([reward], device=system)
        done = terminated or truncated
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(obs, dtype=torch.float32, device=system).unsqueeze(0)

        memory.push(state, action, next_state, reward)
        state = next_state
        if i % 5 == 0:
            optimize_ddqn()
            optimise_target_network(tau=0.001)

        if i % 100 == 0:
            clean_mem()
        if done:
            break
        
    episode_rewards.append(total_reward)
    print(total_reward)

    clean_mem()

print('Done Training')

if system == torch.device("cuda"):
    torch.save(primary_network.state_dict(), 'mario_DDQN_full.pth')
else:
    torch.save(primary_network.state_dict(), 'mario_DDQN_short.pth')

Starting Episode  1
1660.0
Starting Episode  2
1468.0
Starting Episode  3
1179.0
Starting Episode  4
1555.0
Starting Episode  5
1574.0
Starting Episode  6
2017.0
Starting Episode  7
2446.0
Starting Episode  8
2387.0
Starting Episode  9
1866.0
Starting Episode  10
1575.0
Starting Episode  11
1508.0
Starting Episode  12
1712.0
Starting Episode  13
3243.0
Starting Episode  14
2341.0
Starting Episode  15
2002.0
Starting Episode  16
1713.0
Starting Episode  17
1469.0
Starting Episode  18
1399.0
Starting Episode  19
1957.0
Starting Episode  20
1484.0
Starting Episode  21
2606.0
Starting Episode  22
2157.0
Starting Episode  23
1558.0
Starting Episode  24


KeyboardInterrupt: 

In [14]:
# Cleanup ready for A2C
del primary_network, target_network, optimiser, memory, episodes
clean_mem()

# /// A2C Algorithm ///

To save time, we will use some of the same classes and functions from DQN. Make sure you run all of the prior code (except the training loop for DQN).

In [ ]:
steps_passed = 0
episode_rewards = []
episode_steps = []
gamma = 0.99
a2c_update = 0.001

if system == torch.device("cuda"):
    episodes = 600
else:
    episodes = 30


class CriticNetwork(nn.Module):

    def __init__(self, shape, actions_n):
        super(CriticNetwork, self).__init__()
        self.convolution = nn.Sequential( # combined convolutional layers
        nn.Conv2d(shape[0],32, kernel_size=8, stride=4),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=4, stride=2),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, stride=1),
        nn.ReLU()
        )

        output_size = self._get_conv_out(shape)
        self.actor = nn.Sequential(  
            nn.Linear(output_size, 128),
            nn.ReLU(),
            nn.Linear(128, actions_n),
            nn.ReLU()
        )
        self.critic = nn.Sequential(
            nn.Linear(output_size, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def _get_conv_out(self, shape):
        with torch.no_grad():
            blank = torch.zeros(1, *shape)
            out = self.convolution(blank)
            return int(np.prod(out.size())) 

    def get_values(self, x): # for debugging
        x = self.convolution(x)
        actor = self.actor(x).view(x.size(0), -1)
        critic = self.critic(x).view(x.size(0), -1)
        return actor, critic
    
    def forward(self, x):
        x = self.convolution(x).view(x.size(0), -1)
        probs = nn.Softmax(dim=1)(self.actor(x))
        state_values = self.critic(x)
        return probs, state_values


a2c_model = CriticNetwork(obs.shape, action_n).to(system)
a2c_optimiser = opt.Adam(a2c_model.parameters(), lr=a2c_update)

clean_mem()
for episode_n in range(episodes):
    print("Starting Episode ", episode_n+1)
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=system).unsqueeze(0)
    total_reward = 0
    probability_logs = []
    rewards = []
    values = []
    for i in range(20000):  # or count():
        action_probability, value = a2c_model(state)
        action = torch.multinomial(action_probability, 1).item()
        prob_log = torch.log(action_probability.squeeze(0)[action])
        obs, reward, terminated, truncated, info = env.step(action)
        next_state = torch.tensor(obs, dtype=torch.float32, device=system).unsqueeze(0)
        probability_logs.append(prob_log)
        total_reward += reward
        rewards.append(reward)
        values.append(value)
        done = terminated or truncated
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(obs, dtype=torch.float32, device=system).unsqueeze(0)
        state = next_state

        if i % 100 == 0:
            clean_mem()

        if done:
            break

    episode_steps.append(i+1)
    episode_rewards.append(total_reward)
    print(total_reward)


    returns = []
    discounted_return = 0
    for reward in reversed(rewards):
        discounted_return = reward + gamma * discounted_return
        returns.insert(0, discounted_return)
    returns = torch.tensor(returns, dtype=torch.float32, device=system)

    probability_logs = torch.stack(probability_logs)
    values = torch.cat(values).squeeze()
    advantage = returns - values

    actor_loss = -(probability_logs * advantage.detach()).mean()
    critic_loss = advantage.pow(2).mean()

    loss = actor_loss + critic_loss
    a2c_optimiser.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(a2c_model.parameters(), 0.5)
    a2c_optimiser.step()

    clean_mem()

print('Done Training')

if system == torch.device("cuda"):
    torch.save(a2c_model.state_dict(), 'mario_A2C_full.pth')
else:
    torch.save(a2c_model.state_dict(), 'mario_A2C_short.pth')

Starting Episode  1
610.0
Starting Episode  2
493.0


In [14]:
env.close()
del a2c_model, a2c_optimiser, a2c_update, episodes, env

NameError: name 'a2c_model' is not defined

## Run each model on the game

In [ ]:
env = env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode='human')
env = JoypadSpace(env, actions.SIMPLE_MOVEMENT)
env = GreyScale(env)
env = SmallBox84(env)
env = Transpose(env)
env = Normalise(env)
obs, info = env.reset()

dqn = DQN_full(obs.shape, env.action_space.n).to(system)
if cuda():
    loaded = torch.load("mario_DQN_full.pth", map_location=system)
    if isinstance(loaded, dict) and 'model_state_dict' in loaded:
        dqn.load_state_dict(loaded['model_state_dict'])
    else:
        dqn.load_state_dict(loaded)
else:
    loaded = torch.load("mario_DQN_short.pth", map_location=system)
    if isinstance(loaded, dict) and 'model_state_dict' in loaded:
        dqn.load_state_dict(loaded['model_state_dict'])
    else:
        dqn.load_state_dict(loaded)
dqn.eval()

done = None
state, info = env.reset()
'''
if True:
    for step in range(10000):
        state = torch.tensor(state, dtype=torch.float32, device=system).unsqueeze(0)
        with torch.no_grad():
            q_values = dqn(state)
            action = q_values.argmax().item()
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        state = next_state
        if done:
            break
'''
env.close()



c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\gym\envs\registration.py:556: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  f"The environment {id} is out of date. You should consider "
c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\gym\envs\registration.py:628: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  f"The environment creator metadata doesn't include `render_modes`, contains: {list(env_creator.metadata.keys())}"


(84, 84, 240)
